## MQTT Subscriber

In this tutorial, I would like to tell how to implement a MQTT subscriber, which subscribes for a certain type of data from MQTT broker. 

I would like to create a hypothetical smart city, which clients publish below data:

```
- temperature
    - celcius
    - fahrenheit
- air pollution
    - CO level
    - SO2 level
- traffic speed
    - km/h
```

Subscribers only work on determined data types, a single purpose clients to collect data and do what ever they want. Like, show on the map, analyze etc.

Our topic structure is as shown:

```
<system_name>/<city_id>/<street_id>/<sensor_data>/<sub_data>
```

We want our subscriber to collect certain data for its own purpose. Since our topic structure is known by all systems, clients can be configured to parse data and collect certain information.

Using the topic info, a subscriber can identify id of city, street and sensor data types. 

This information can also be embedded into payload, that's the choice of design. 



In [1]:
# let's check if paho-mqtt has been installed
!pip install paho-mqtt

In [2]:
# import paho client
import paho.mqtt.client as mqtt

In [3]:
# Define callback functions for Subscriber
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    
def on_disconnect(client, userdata, rc):
    print("Connected with result code " + str(rc))
    
    
def on_message(client, userdata, msg):
    message = msg.payload.decode("utf-8")

    message_data = message.split('\t')
    timestamp_utc = message_data[0]
    data = message_data[1]
    
    topic_data = msg.topic.split('/')
    pollution_type = topic_data[len(topic_data) - 1]
    
    if pollution_type == 'no2_level':
        print('City: {}, Street {}, NO2 Level {} ug/m3 | Timestamp: {}'.format(topic_data[1], topic_data[2], message_data[1], timestamp_utc))
    elif pollution_type == 'so2_level':
        print('City: {}, Street {}, SO2 Level {} ug/m3 | Timestamp: {}'.format(topic_data[1], topic_data[2], message_data[1], timestamp_utc))
    else:
        print('City: {}, Street {}, Sensor Type {}, Data {} | Timestamp: {} '.format(topic_data[1], topic_data[2], topic_data[len(topic_data) - 1],message_data[1] , timestamp_utc))

In [4]:
mqtt_server_ip = "127.0.0.1"
server_port = 1883
tls_server_port = 8883

# uniqie client id
client_id = 'air_pollution_analyzer'

In [ ]:
system_name = "hypo_city"
air_pollution = 'air_pollution'

client = mqtt.Client(client_id)

username = 'subscriber1'
password = 'simple_password'

# set username password
client.username_pw_set(username, password=password)

client.tls_set('ca.crt')

client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message

client.connect(mqtt_server_ip, tls_server_port, keepalive=60)

# I will get, all cities and all cities streets 
subscribe_topic = '#' #system_name + '/+/+/+/+'

client.subscribe(subscribe_topic, qos=0)

# start loop forever to subscribe wanted topic
client.loop_forever()

Connected with result code 0
City: city_01, Street street_01, Sensor Type celcius, Data 23.22003860342444 | Timestamp: 2018-08-05 16:16:45.103554 
City: city_01, Street street_01, NO2 Level 8.853369352523995 ug/m3 | Timestamp: 2018-08-05 16:16:45.103554
City: city_01, Street street_01, Sensor Type traffic_speed, Data 18.704987264387817 | Timestamp: 2018-08-05 16:16:45.103554 
City: city_01, Street street_01, Sensor Type fahrenheit, Data 43.56886808597102 | Timestamp: 2018-08-05 16:16:45.103554 
City: city_01, Street street_01, SO2 Level 9.525179967460652 ug/m3 | Timestamp: 2018-08-05 16:16:45.104564
